In [1]:
import copy
from sklearn.model_selection import LeaveOneOut
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import pretrainedmodels
import warnings
import gc
import pandas as pd
import os
warnings.filterwarnings("ignore")

from utils.utils_data_load import *
from train.train_functions import *

In [2]:
data_dir = '../data/'
model_path = '../results/'
experiment = 'resnet34_1'
description = 'center crop resnet 34'
model_path = os.path.join(model_path, experiment)
if not os.path.exists(model_path):
    os.makedirs(model_path)
    
use_gpu = torch.cuda.is_available()

In [3]:
with open(os.path.join(model_path, 'description.txt'), 'w') as f:
    f.write("%s\n" % description)

In [4]:
from albumentations.pytorch import ToTensor
from albumentations import (Compose, CenterCrop, VerticalFlip,
                            HorizontalFlip, HueSaturationValue, Resize)

In [5]:
from albumentations.pytorch import ToTensor
from albumentations import (Compose, CenterCrop, VerticalFlip,
                            HorizontalFlip, HueSaturationValue, Resize)


In [6]:
from utils.utils_loader import ImageFolder

In [7]:
data_transforms = {
    'train': Compose([
    CenterCrop(224, 224), 
    VerticalFlip(),
    HorizontalFlip(),
    HueSaturationValue(),
    ToTensor()
]),
    'val': Compose([
    CenterCrop(224, 224), 
    ToTensor()
]),
    'test': Compose([
    CenterCrop(224, 224), 
    ToTensor()
]),
}
def change_classes(x):
    return 1 - x

target_transform = change_classes

In [8]:
from matplotlib import pyplot as plt

In [9]:
from PIL import Image
from torchvision import datasets, models, transforms

In [10]:
from sklearn.metrics import roc_curve, precision_recall_curve, f1_score

In [11]:
import torch.nn.functional as F

In [13]:
def sigmoid(x):
    return 1 / (1+np.exp(-1*x))

loo = LeaveOneOut()
folds = np.array(['fold_0','fold_1','fold_2', 'fold_3'])
#models = []
results = pd.DataFrame()
device  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for tr, vl in loo.split(folds):
    model_ft = pretrainedmodels.__dict__['se_resnext50_32x4d'](num_classes=1000, pretrained='imagenet')

    model_ft.last_linear = nn.Linear(2048, 20)


    if use_gpu:
        model_ft = model_ft.to(device)
    criterion = nn.BCEWithLogitsLoss(reduction='sum')

    params_to_train = model_ft.parameters()
    optimizer_ft = torch.optim.Adam(params_to_train)
    plat_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft,
                                                           'min', patience=3,
                                                           factor=0.999, verbose=True)

    train_folds = folds[tr]
    val_folds = folds[vl]
    test_data = ['test']

    dataloaders, dataset_sizes, class_names = loader(data_transforms, train_folds, val_folds,
                                                         test_data, data_dir, bs=64, target_transform=target_transform)
    
    model_ft, best_score = train_model(model_ft, criterion, optimizer_ft,
                               plat_lr_scheduler,
                               dataset_sizes=dataset_sizes,
                               model_path=model_path,
                               dataloaders=dataloaders,
                               device=device, 
                               num_epochs=3, fold_name=folds[vl][0])
    torch.save(model_ft.state_dict(),
               os.path.join(model_path, 'val_' + folds[vl][0] + '_f1_05_' + str(best_score).replace('.', '')))
    del criterion, optimizer_ft, plat_lr_scheduler
    torch.cuda.empty_cache()
    gc.collect()


  0%|          | 0/141 [00:00<?, ?it/s]

VAL data fold_0
Epoch 0/2
----------


  6%|▌         | 8/141 [00:03<01:05,  2.02it/s]


KeyboardInterrupt: 